# Exploratory Data Analysis

## Imports

In [19]:
import os
from dotenv import load_dotenv

import polars as pl

## Config

In [20]:
# Load environment variables
load_dotenv()

preprocessed_dir = os.getenv('PREPROCESSED_DATA_DIR', 'data/preprocessed')

In [21]:
# Change working directory
os.chdir("/home/mle-user/mle_projects/mle-project-sprint-4")

## Preprocessed Data Loading

In [22]:
# ---------- Load datasets ---------- #
items_path = os.path.join(preprocessed_dir, 'items.parquet')
events_path = os.path.join(preprocessed_dir, 'events.parquet')
tracks_catalog_path = os.path.join(preprocessed_dir, 'tracks_catalog_clean.parquet')

# Load preprocessed data
items = pl.read_parquet(items_path)
events = pl.read_parquet(events_path)
tracks_catalog_clean = pl.read_parquet(tracks_catalog_path)

print(f'Data loaded')
print(f'Items: {items.shape}')
print(f'Events: {events.shape}')
print(f'Tracks catalog: {tracks_catalog_clean.shape}')

Data loaded
Items: (5574943, 9)
Events: (161723765, 4)
Tracks catalog: (804916, 6)


## EDA

In [23]:
# ---------- Check data summary ---------- #

def data_summary(df: pl.DataFrame, name: str):
    '''
    Display a quick overview of a Polars DataFrame.
    '''
    print('\n' + '='*50)
    print(f'\n{name}')
    print('='*50)
  
    # Sample rows
    print('\nSample rows:')
    display(df.head())

    # Shape
    rows, cols = df.shape
    print(f'\nShape: {rows:,} rows x {cols} columns')
    
    # Data info
    print('\nSummary for numeric columns:')
    display(df.describe())
   
    # Column info
    print('\nColumn names and types:')
    for col in df.columns:
        print(f'  {col}: {df[col].dtype}')
    
    # Missing values
    print('\nMissing values:')
    display(df.null_count())

In [24]:
# ---------- Items data summary ---------- #
data_summary(items, 'items')



items

Sample rows:


track_id,artist_id,album_id,genre_id,track_clean,track_group_id,artist_clean,album_clean,genre_clean
i64,i64,i64,i64,str,i64,str,str,str
88022,9322,112192,23,"""The Second Waltz Op 99A""",88022,"""Дмитрии Дмитриевич Шостакович""","""Talents""","""Classical"""
125572,45594,8073477,37,"""Groove Is In The Heart""",125572,"""Deee Lite""","""90S Dance""","""Disco"""
71240,451,14689677,74,"""Let S Get Retarded""",71240,"""Black Eyed Peas""","""Поп Мотивация""","""Rnb"""
105008,41075,10126,2,"""Время Есть А Денег Нет""",105008,"""Кино""","""45""","""Rusrock"""
80332,24301,10347119,59,"""The Boys Are Back In Town""",80332,"""Thin Lizzy""","""Work From Home Rock Edition""","""Hardrock"""



Shape: 5,574,943 rows x 9 columns

Summary for numeric columns:


statistic,track_id,artist_id,album_id,genre_id,track_clean,track_group_id,artist_clean,album_clean,genre_clean
str,f64,f64,f64,f64,str,f64,str,str,str
"""count""",5.574943e6,5.574943e6,5.574943e6,5.574943e6,"""5574943""",5.574943e6,"""5574943""","""5574943""","""5574943"""
"""null_count""",0.0,0.0,0.0,0.0,"""0""",0.0,"""0""","""0""","""0"""
"""mean""",2.5712e7,1.7933e6,6.0413e6,79.068111,null,2.5698e7,null,null,null
"""std""",2.2864e7,2.2239e6,5.4586e6,119.351925,null,2.2862e7,null,null,null
"""min""",26.0,4.0,3.0,1.0,"""""",26.0,"""""","""""","""African"""
"""25%""",4.553699e6,49430.0,2.529826e6,19.0,null,4.547738e6,null,null,null
"""50%""",2.353175e7,665321.0,3.362318e6,49.0,null,2.3531749e7,null,null,null
"""75%""",3.5849872e7,3.343208e6,9.755566e6,74.0,null,3.5837594e7,null,null,null
"""max""",1.01495927e8,1.6102782e7,2.1458968e7,1542.0,"""Ꞌcosmicꞌ M4A""",1.01495927e8,"""龍胆紫 Purplesoul""","""Ꞌcosmicꞌ M4A""","""Work"""



Column names and types:
  track_id: Int64
  artist_id: Int64
  album_id: Int64
  genre_id: Int64
  track_clean: String
  track_group_id: Int64
  artist_clean: String
  album_clean: String
  genre_clean: String

Missing values:


track_id,artist_id,album_id,genre_id,track_clean,track_group_id,artist_clean,album_clean,genre_clean
u32,u32,u32,u32,u32,u32,u32,u32,u32
0,0,0,0,0,0,0,0,0


In [25]:
# ---------- Events data summary ---------- #
data_summary(events, 'events')



events

Sample rows:


user_id,track_id,listen_count,last_listen
i32,i32,u32,date
2057,38551191,1,2022-11-21
2057,61638507,1,2022-12-09
2058,55860184,1,2022-11-01
2063,732388,1,2022-04-22
2063,44030455,1,2022-09-09



Shape: 161,723,765 rows x 4 columns

Summary for numeric columns:


statistic,user_id,track_id,listen_count,last_listen
str,f64,f64,f64,str
"""count""",1.61723765e8,1.61723765e8,1.61723765e8,"""161723765"""
"""null_count""",0.0,0.0,0.0,"""0"""
"""mean""",687114.400837,3.7348e7,1.0,"""2022-08-29 12:51:43.611870"""
"""std""",396876.87637,2.7485e7,0.0,null
"""min""",0.0,26.0,1.0,"""2022-01-01"""
"""25%""",343000.0,1.2712954e7,1.0,"""2022-07-01"""
"""50%""",687433.0,3.6817068e7,1.0,"""2022-09-15"""
"""75%""",1.030401e6,5.8968902e7,1.0,"""2022-11-10"""
"""max""",1.374582e6,1.01495927e8,1.0,"""2022-12-31"""



Column names and types:
  user_id: Int32
  track_id: Int32
  listen_count: UInt32
  last_listen: Date

Missing values:


user_id,track_id,listen_count,last_listen
u32,u32,u32,u32
0,0,0,0


Data fully cleaned and ready for futher analysis.

In [26]:
#---------- Top tracks by popularity ---------- #
print('\n' + '='*50)
print(f'\nTop Tracks by popularity (number of listens)')
print('='*50)

top_tracks_by_listen_number = (
    events
        .group_by('track_id')
        .agg(pl.sum('listen_count').alias('total_listen_count'))
        .join(tracks_catalog_clean.select(['track_id', 'track_clean']), on='track_id', how='left')
        .sort('total_listen_count', descending=True)
        .head(10)
)

display(top_tracks_by_listen_number)



Top Tracks by popularity (number of listens)


track_id,total_listen_count,track_clean
i32,u32,str
53404,111062,"""Smells Like Teen Spirit"""
33311009,105666,"""Believer"""
35505245,98962,"""I Got Love"""
24692821,85602,"""Way Down We Go"""
65851540,85173,"""Юность"""
795836,84976,"""Shape Of My Heart"""
32947997,84973,"""Shape Of You"""
45499814,83648,"""Life"""
60292250,81977,"""Blinding Lights"""


In [27]:
# Top genres by popularity
print('\n' + '='*50)
print(f'\nTop 5 Genres by Listening Number')
print('='*50)

# Aggregate events by track_id (reduces size)
events_by_track = (
    events
        .lazy()
        .group_by('track_id')
        .agg(pl.sum('listen_count').alias('track_listen_count'))
        .collect()
)

# Get unique track-genre mapping
track_genres = items.select(['track_id', 'genre_clean']).unique(['track_id', 'genre_clean'])

# Join and aggregate
genres_by_listen_count = (
    events_by_track
        .join(track_genres, on='track_id', how='left')
        .group_by('genre_clean')
        .agg(pl.sum('track_listen_count').alias('total_listen_count'))
        .sort('total_listen_count', descending=True)
)

top_5_genres = genres_by_listen_count.head(5)
display(top_5_genres)



Top 5 Genres by Listening Number


genre_clean,total_listen_count
str,u32
"""Pop""",47681498
"""Rap""",30921736
"""Ruspop""",22028416
"""Allrock""",21320352
"""Rusrap""",20450969


In [28]:
# Tracks that haven't been listened to by anybody
print('\n' + '='*50)
print(f'\nTracks that havent been listened to')
print('='*50)

# Get set of listened track ids
listened_track_ids = set(events['track_id'].unique())

# Get unique tracks from items
unique_tracks = (
    items
        .select(['track_id', 'track_clean', 'artist_clean', 'album_clean', 'genre_clean'])
        .unique('track_id')
)

# Filter out listened tracks
unlistened_tracks = (
    unique_tracks
        .filter(~pl.col('track_id')
        .is_in(list(listened_track_ids)))
)

print(f'Number of unlistened tracks: {unlistened_tracks.height:,}')
display(unlistened_tracks.head(10))



Tracks that havent been listened to
Number of unlistened tracks: 49


track_id,track_clean,artist_clean,album_clean,genre_clean
i64,str,str,str,str
97264237,"""Рождество""","""Арт Группа Largo""","""Рождество""","""Spiritual"""
73459486,"""Lord Of The Flies""","""Elton John""","""Jewel Box""","""Pop"""
88466656,"""Last Horizon""","""Brian May""","""Back To The Light""","""Allrock"""
56166858,"""Панк Туристы""","""Скамеика Запасных""","""Новые Грехи Старые Традиции""","""Punk"""
61606834,"""В Любои Момент Могу Бросить""","""Вбензинеестьоблака""","""Все Что Вы Хотели Но Боялись Е…","""Rusrap"""
57627360,"""Statues""","""Death From Above 1979""","""Outrage Is Now""","""Alternative"""
46786298,"""Heresy Blind""","""Nine Inch Nails""","""Closer To God""","""Industrial"""
70050701,"""Symphonic Suite Kiki S Deliver…","""Joe Hisaishi""","""Symphonic Suite Kiki S Deliver…","""Modern"""
77751634,"""Gypsy Drum Solo""","""Dio""","""Holy Diver""","""Classicmetal"""
